In [14]:
from random import shuffle
import glob

shuffle_data = True  # shuffle the addresses

hdf5_path = 'Covid19Seq_microDoppler128x128.hdf5'  # file path for the created .hdf5 file

train_path = '/media/m/FantomHD/COVID19 Sequential/Output/*/microDoppler/77ghz/Front/Cropped/colored/*/*.png' # the original data path
print(train_path)

# get all the image paths 
addrs = glob.glob(train_path)
print(len(addrs))


/media/m/FantomHD/COVID19 Sequential/Output/*/microDoppler/77ghz/Front/Cropped/colored/*/*.png
952


In [15]:
labels = []
for i in addrs:
#     if 'Cane' in i:
#         labels.append(0)
#     elif 'Crawling' in i:
#         labels.append(1)
#     elif 'Creeping' in i:
#         labels.append(2)
#     elif 'Crutch' in i:
#         labels.append(3)
#     elif 'Falling Front' in i:
#         labels.append(4)
#     elif 'Limping' in i:
#         labels.append(5)
#     elif 'Running' in i:
#         labels.append(6)
#     elif 'Tripod' in i:
#         labels.append(7)
#     elif 'Walker' in i:
#         labels.append(8)
#     elif 'Walking' in i:
#         labels.append(9)
#     elif 'Wheeled Chair' in i:
#         labels.append(10)

#     elif 'YD' in i:
#         labels.append(11)

########################
#     if 'towards' in i:
#         labels.append(0)
#     elif 'away' in i:
#         labels.append(1)
#     elif 'picking' in i:
#         labels.append(2)
#     elif 'bending' in i:
#         labels.append(3)
#     elif 'sitting' in i:
#         labels.append(4)
#     elif 'kneeling' in i:
#         labels.append(5)
#     elif 'crawling' in i:
#         labels.append(6)
#     elif 'toes' in i:
#         labels.append(7)
#     elif 'limping' in i:
#         labels.append(8)
#     elif 'short' in i:
#         labels.append(9)
#     elif 'scissor' in i:
#         labels.append(10)
##############
    if 'no movement' in i:
        labels.append(0)
    elif 'walking' in i:
        labels.append(1)
    elif 'sitting' in i:
        labels.append(2)
    elif 'standing' in i:
        labels.append(3)
    elif 'folding laundry' in i:
        labels.append(4)
    elif 'ironing' in i:
        labels.append(5)
    elif 'you' in i:
        labels.append(6)
    elif 'hello' in i:
        labels.append(7)
    elif 'car' in i:
        labels.append(8)
    elif 'push' in i:
        labels.append(9)


In [16]:
print(len(labels))

952


In [17]:
"""
# label the data as 0=cat, 1=dog

labels = [0 if 'towards' in else 1 for addr in addrs] # extra is included in non-vehicles name 
          
    
"""

# shuffle data
if shuffle_data:
    c = list(zip(addrs, labels)) # use zip() to bind the images and labels together
    shuffle(c)
 
    (addrs, labels) = zip(*c)  # *c is used to separate all the tuples in the list c,  
                               # "addrs" then contains all the shuffled paths and 
                               # "labels" contains all the shuffled labels.
                               
# Divide the data into 80% for train and 20% for test
# train_addrs = addrs[0:int(0.7*len(addrs))]
# train_labels = labels[0:int(0.7*len(labels))]
train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]

# valid_addrs = addrs[int(0.7*len(addrs)):int(0.8*len(addrs))]
# valid_labels = labels[int(0.7*len(labels)):int(0.8*len(addrs))]

test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]

print(len(train_addrs))
# print(len(valid_addrs))
print(len(test_addrs))
print(len(train_labels))
# print(len(valid_labels))
print(len(test_labels))

##################### second part: create the h5py object #####################
import numpy as np
import h5py, cv2
im_width = 128
im_height = 128
train_shape = (len(train_addrs), im_height, im_width, 3)
# valid_shape = (len(valid_addrs), im_width, im_height, 3)
test_shape = (len(test_addrs), im_height, im_width, 3)

# open a hdf5 file and create earrays 
f = h5py.File(hdf5_path, mode='w')

# PIL.Image: the pixels range is 0-255,dtype is uint.
# matplotlib: the pixels range is 0-1,dtype is float.
f.create_dataset("train_img", train_shape, np.uint8)
# f.create_dataset("valid_img", valid_shape, np.uint8)  
f.create_dataset("test_img", test_shape, np.uint8)  

# the ".create_dataset" object is like a dictionary, the "train_labels" is the key. 
f.create_dataset("train_labels", (len(train_addrs),), np.uint8)
f["train_labels"][...] = train_labels

# f.create_dataset("valid_labels", (len(valid_addrs),), np.uint8)
# f["valid_labels"][...] = valid_labels

f.create_dataset("test_labels", (len(test_addrs),), np.uint8)
f["test_labels"][...] = test_labels

######################## third part: write the images #########################
import cv2

# loop over train paths
for i in range(len(train_addrs)):
  
    if i % 50 == 0 and i > 1:
        print ('Train data: {}/{}'.format(i, len(train_addrs)) )

    addr = train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    f["train_img"][i, ...] = img[None] 

    # loop over valid paths
# for i in range(len(valid_addrs)):

#     if i % 50 == 0 and i > 1:
#         print ('Valid data: {}/{}'.format(i, len(valid_addrs)) )
        
#     addr = valid_addrs[i]
#     img = cv2.imread(addr)
#     img = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_CUBIC)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     f["valid_img"][i, ...] = img[None]
    
# loop over test paths
for i in range(len(test_addrs)):

    if i % 50 == 0 and i > 1:
        print ('Test data: {}/{}'.format(i, len(test_addrs)) )
        
    addr = test_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    f["test_img"][i, ...] = img[None]

f.close()


761
191
761
191
Train data: 50/761
Train data: 100/761
Train data: 150/761
Train data: 200/761
Train data: 250/761
Train data: 300/761
Train data: 350/761
Train data: 400/761
Train data: 450/761
Train data: 500/761
Train data: 550/761
Train data: 600/761
Train data: 650/761
Train data: 700/761
Train data: 750/761
Test data: 50/191
Test data: 100/191
Test data: 150/191


In [18]:
f.close()

## Read .h5 file and verify

In [19]:
dataset = h5py.File(hdf5_path, "r")

test_labels = np.array(dataset["test_labels"])
# valid_labels = np.array(dataset["valid_labels"])
train_labels = np.array(dataset["train_labels"])
train_img = np.array(dataset["train_img"])
# valid_img = np.array(dataset["valid_img"])
test_img = np.array(dataset["test_img"])
dataset.close()
print(len(train_labels))
# print(len(valid_labels))
print(len(test_labels))
cv2.imshow('Sample', test_img[5])



761
191
